In [2]:
%load_ext pycodestyle_magic

In [3]:
%pycodestyle_on

In [10]:
import numpy as np
import itertools
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from task01_romskiydg_model import ElasticNetRegressor
from sklearn.metrics import r2_score


X, y = load_boston(return_X_y=True)
trainX, testX, trainY, testY = train_test_split(X, y, random_state=42,
                                                test_size=0.2)
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

(404, 13) (404,) (102, 13) (102,)


В будущем можно добавить совместимость с GridSearchCV, для удобства поиска параметров

In [13]:
trainX_1 = trainX[::4]
trainX_2 = trainX[1::4]
trainX_3 = trainX[2::4]
trainX_4 = trainX[3::4]
trainY_1 = trainY[::4]
trainY_2 = trainY[1::4]
trainY_3 = trainY[2::4]
trainY_4 = trainY[3::4]
trainX_csv = np.empty((4, 303, 13))
trainY_csv = np.empty((4, 303))
testX_csv = np.empty((4, 101, 13))
testY_csv = np.empty((4, 101))

trainX_csv[0] = np.vstack((trainX_1, trainX_2, trainX_3))
testX_csv[0] = trainX_4
trainY_csv[0] = np.hstack((trainY_1, trainY_2, trainY_3))
testY_csv[0] = trainY_4

trainX_csv[1] = np.vstack((trainX_1, trainX_2, trainX_4))
testX_csv[1] = trainX_3
trainY_csv[1] = np.hstack((trainY_1, trainY_2, trainY_4))
testY_csv[1] = trainY_3

trainX_csv[2] = np.vstack((trainX_1, trainX_4, trainX_3))
testX_csv[2] = trainX_2
trainY_csv[2] = np.hstack((trainY_1, trainY_4, trainY_3))
testY_csv[2] = trainY_2

trainX_csv[3] = np.vstack((trainX_4, trainX_2, trainX_3))
testX_csv[3] = trainX_1
trainY_csv[3] = np.hstack((trainY_4, trainY_2, trainY_3))
testY_csv[3] = trainY_1

In [12]:
sets_for_iterating = [
    [1000, 2000],
    [0.001, 0.0001],
    [0.01, 0.001],
    [5, 10]
]
best_score = 5.
for element in itertools.product(*sets_for_iterating):
    score = 0
    for i in range(4):
        csv_regressor = ElasticNetRegressor(n_epoch=element[0],
                                            alpha=element[1],
                                            delta=element[2],
                                            batch_size=element[3])
        csv_regressor.fit(trainX_csv[i], trainY_csv[i])
        prediction = csv_regressor.predict(testX_csv[i])
        score += csv_regressor.score(testY_csv[i], prediction)
    if score > best_score or best_score == 5.:
        best_score = score
        best_param = element

my_regressor = ElasticNetRegressor(n_epoch=best_param[0],
                                   alpha=best_param[1],
                                   delta=best_param[2],
                                   batch_size=best_param[3])
my_regressor.fit(trainX, trainY)
prediction = my_regressor.predict(testX)
R2_score = my_regressor.score(testY, prediction)
print(R2_score, best_score/4)
print(best_param)

0.6261625419030828 0.6681313423183404
(2000, 0.001, 0.01, 10)
